# 0. Import Dependencies

In [67]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import (LSTM, Dense, Concatenate, Attention, Dropout, Softmax,
                                     Input, Flatten, Activation, Bidirectional, Permute, multiply, 
                                     ConvLSTM2D, MaxPooling3D, TimeDistributed, Conv2D, MaxPooling2D)

from scipy import stats

# disable some of the tf/keras training warnings 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(1)

# suppress untraced functions warning
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# 1. Keypoints using MP Pose

In [71]:
# Pre-trained pose estimation model from Google Mediapipe

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [72]:
def mediapipe_detection(image, model):
    """
    This function detects human pose estimation keypoints from webcam footage
    
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [73]:
def draw_landmarks(image, results):
    """
    This function draws keypoints and landmarks detected by the human pose estimation model
    
    """
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )

In [5]:
cap = cv2.VideoCapture(0) # camera object
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 

# Set and test mediapipe model using webcam
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        #Read feed
        ret, frame = cap.read()   
        # Make detection
        image, results = mediapipe_detection(frame, pose)
        
        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Render detections
        draw_landmarks(image, results)               
        
        # Display frame on screen
        cv2.imshow('OpenCV Feed', image)
        
        # Exit / break out logic
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# 2. Extract Keypoints

In [74]:
# Recollect and organize keypoints from the test
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [75]:
# 33 landmarks with 4 values (x, y, z, visibility)
num_landmarks = len(landmarks)
num_values = len(test)
num_input_values = num_landmarks*num_values
print(num_landmarks,num_values,num_input_values)

33 4 132


In [57]:
### This is an example of what we would use as an input into our AI models
#pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

In [76]:
def extract_keypoints(results):
    """
    Processes and organizes the keypoints detected from the pose estimation model 
    to be used as inputs for the exercise decoder models
    
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

# 3. Setup Folders for Collection

In [78]:
#### RUN THIS WHEN USING PRE EXISTING DATA

# Path for exported data, numpy arrays
DATA_PATH = os.path.join(os. getcwd(),'data') 
print(DATA_PATH)

# make directory if it does not exist yet
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Actions/exercises that we try to detect
actions = np.array(['curl','press','squat','butbrdige','leglift'])
colors = [(245,117,16), (117,245,16),(16,117,245),(127, 0, 255),(255, 255, 0)]
num_classes = len(actions)

# How many videos worth of data
no_sequences = 1

# Videos are going to be this many frames in length
sequence_length = FPS*1

# Folder start
# Change this to collect more data and not lose previously collected data
start_folder = 1

/Users/tsaiajay/Desktop/CP575/Exercise_Recognition_AI-main/data


In [62]:
#DONT RUN THIS IF YOU RUN THE ABOVE CELL

##### RUN THIS when NEED TO ADD NEW EXCERSICE OR NEW DAta

DATA_PATH = os.path.join(os. getcwd(),'data') 
print(DATA_PATH)

# make directory if it does not exist yet
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Actions/exercises that we try to detect
actions = np.array(['butbrdige'])
colors =[(127, 0, 255)]
num_classes = len(actions)

# How many videos worth of data
no_sequences = 1

# Videos are going to be this many frames in length
sequence_length = FPS*1

# Folder start
# Change this to collect more data and not lose previously collected data
start_folder = 120

/Users/tsaiajay/Desktop/CP575/Exercise_Recognition_AI-main/data


In [63]:
#DONT RUN IF YOU HAVE DATA EXISTS

#RUN IF YOU NEED TO GET MORE data

# Build folder paths
for action in actions:     
    for sequence in range(start_folder,no_sequences+start_folder):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))  
        except:
            pass

# 4. Collect Keypoint Values for Training and Testing

In [65]:
#DONT RUN IF YOU ALREADY HAVE DATA 

# Collect Training Data

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6) as pose:
    # Loop through actions
    for idx, action in enumerate(actions):
        # Loop through sequences (i.e., videos)
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length (i.e, sequence length)
            for frame_num in range(sequence_length):
                # Read feed
                ret, frame = cap.read()
                
                # Make detection
                image, results = mediapipe_detection(frame, pose)

                # Extract landmarks
                try:
                    landmarks = results.pose_landmarks.landmark
                except:
                    print("INTO EXCEPT")
                    pass
                
                # Render detections
                draw_landmarks(image, results) 
                
                # Apply visualization logic
                if frame_num == 0: # If first frame in sequence, print that you're starting a new data collection and wait 500 ms
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(800)
                else: 
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # Export keypoints (sequence + pose landmarks)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [66]:
cap.release()
cv2.destroyAllWindows()

# 5. Preprocess Data and Create Labels/Features

In [80]:
label_map = {label:num for num, label in enumerate(actions)}

In [81]:
sequences, labels = [], []
for action in actions:
    for sequence in os.listdir(os.path.join(DATA_PATH, action)):
        # Check if the file is a directory and not a hidden file
        if os.path.isdir(os.path.join(DATA_PATH, action, sequence)) and not sequence.startswith('.'):
            window = []
            for frame_num in range(sequence_length):
                # LSTM input data
                res = np.load(os.path.join(DATA_PATH, action, sequence, "{}.npy".format(frame_num)))
                window.append(res)
                
            sequences.append(window)
            labels.append(label_map[action])

In [82]:
# Make sure first dimensions of arrays match
X = np.array(sequences)
y = to_categorical(labels).astype(int)
print(X.shape, y.shape)

(585, 29, 132) (585, 5)


In [83]:
# Split into training, validation, and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)
print(X_train.shape, y_train.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=15/90, random_state=2)

(526, 29, 132) (526, 5)


# 6. Build and Train Neural Networks

In [84]:
# Callbacks to be used during neural network training 
es_callback = EarlyStopping(monitor='val_loss', min_delta=5e-4, patience=10, verbose=0, mode='min')
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=0, mode='min')
chkpt_callback = ModelCheckpoint(filepath=DATA_PATH, monitor='val_loss', verbose=0, save_best_only=True, 
                                 save_weights_only=False, mode='min', save_freq=1)

# Optimizer
#opt = tf.keras.optimizers.Adam(learning_rate=0.01)
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)


# some hyperparamters
batch_size = 32
max_epochs = 500

## 6a. LSTM

In [85]:
# Set up Tensorboard logging and callbacks
NAME = f"ExerciseRecognition-LSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]

In [86]:
lstm = Sequential()
lstm.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(sequence_length, num_input_values)))
lstm.add(LSTM(256, return_sequences=True, activation='relu'))
lstm.add(LSTM(128, return_sequences=False, activation='relu'))
lstm.add(Dense(128, activation='relu'))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(actions.shape[0], activation='softmax'))
print(actions.shape[0])
print(lstm.summary())

5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 29, 128)           133632    
                                                                 
 lstm_1 (LSTM)               (None, 29, 256)           394240    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 750085 (2.86 MB)
Trainable params: 75008

In [87]:
lstm.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
lstm.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/500
14/14 [==============================] - 6s 160ms/step - loss: 3496120.0000 - categorical_accuracy: 0.1872 - val_loss: 33031.3867 - val_categorical_accuracy: 0.1705 - lr: 0.0100
Epoch 2/500
14/14 [==============================] - 1s 103ms/step - loss: 4624.3062 - categorical_accuracy: 0.2192 - val_loss: 1663.7532 - val_categorical_accuracy: 0.2614 - lr: 0.0100
Epoch 3/500
14/14 [==============================] - 1s 105ms/step - loss: 497.6345 - categorical_accuracy: 0.1712 - val_loss: 136.5656 - val_categorical_accuracy: 0.2045 - lr: 0.0100
Epoch 4/500
14/14 [==============================] - 1s 104ms/step - loss: 63.0858 - categorical_accuracy: 0.2534 - val_loss: 77.4944 - val_categorical_accuracy: 0.3409 - lr: 0.0100
Epoch 5/500
14/14 [==============================] - 2s 112ms/step - loss: 35.5240 - categorical_accuracy: 0.3904 - val_loss: 23.0392 - val_categorical_accuracy: 0.4318 - lr: 0.0100
Epoch 6/500
14/14 [==============================] - 1s 105ms/step - loss: 1

## 6b. LSTM + Attention

In [88]:
# Set up Tensorboard logging and callbacks
NAME = f"ExerciseRecognition-AttnLSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]

In [89]:
def attention_block(inputs, time_steps):
    """
    Attention layer for deep neural network
    
    """
    # Attention weights
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    
    # Attention vector
    a_probs = Permute((2, 1), name='attention_vec')(a)
    
    # Luong's multiplicative score
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul') 
    
    return output_attention_mul

In [90]:
HIDDEN_UNITS = 256

# Input
inputs = Input(shape=(sequence_length, num_input_values))

# Bi-LSTM
lstm_out = Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True))(inputs)

# Attention
attention_mul = attention_block(lstm_out, sequence_length)
attention_mul = Flatten()(attention_mul)

# Fully Connected Layer
x = Dense(2*HIDDEN_UNITS, activation='relu')(attention_mul)
x = Dropout(0.5)(x)

# Output
x = Dense(actions.shape[0], activation='softmax')(x)

# Bring it all together
AttnLSTM = Model(inputs=[inputs], outputs=x)
print(AttnLSTM.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 29, 132)]            0         []                            
                                                                                                  
 bidirectional (Bidirection  (None, 29, 512)              796672    ['input_1[0][0]']             
 al)                                                                                              
                                                                                                  
 permute (Permute)           (None, 512, 29)              0         ['bidirectional[0][0]']       
                                                                                                  
 dense_3 (Dense)             (None, 512, 29)              870       ['permute[0][0]']         

In [91]:
AttnLSTM.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
AttnLSTM.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/500
14/14 [==============================] - 7s 210ms/step - loss: 1.5718 - categorical_accuracy: 0.4361 - val_loss: 1.5177 - val_categorical_accuracy: 0.7727 - lr: 1.6000e-05
Epoch 2/500
14/14 [==============================] - 2s 156ms/step - loss: 1.4392 - categorical_accuracy: 0.7306 - val_loss: 1.3444 - val_categorical_accuracy: 0.7500 - lr: 1.6000e-05
Epoch 3/500
14/14 [==============================] - 2s 158ms/step - loss: 1.2201 - categorical_accuracy: 0.7763 - val_loss: 1.0924 - val_categorical_accuracy: 0.7386 - lr: 1.6000e-05
Epoch 4/500
14/14 [==============================] - 2s 157ms/step - loss: 0.9541 - categorical_accuracy: 0.7808 - val_loss: 0.8478 - val_categorical_accuracy: 0.7273 - lr: 1.6000e-05
Epoch 5/500
14/14 [==============================] - 2s 153ms/step - loss: 0.7253 - categorical_accuracy: 0.8356 - val_loss: 0.6492 - val_categorical_accuracy: 0.8636 - lr: 1.6000e-05
Epoch 6/500
14/14 [==============================] - 2s 156ms/step - loss: 0.560

In [92]:
# Model map
models = {
    'LSTM_new': lstm, 
    'LSTM_Attention_new': AttnLSTM, 
}

# 7a. Save Weights

In [93]:
for model_name, model in models.items():
    save_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.save(save_dir)

/Users/tsaiajay/anaconda3/envs/myenv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 7b. Load Weights

In [94]:
# Run model rebuild before doing this
for model_name, model in models.items():
    load_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.load_weights(load_dir)

# 8. Make Predictions

In [95]:
for model in models.values():
    res = model.predict(X_test, verbose=0)   

# 9. Evaluations using Confusion Matrix and Accuracy

In [96]:
eval_results = {}
eval_results['confusion matrix'] = None
eval_results['accuracy'] = None
eval_results['precision'] = None
eval_results['recall'] = None
eval_results['f1 score'] = None

confusion_matrices = {}
classification_accuracies = {}   
precisions = {}
recalls = {}
f1_scores = {} 

## 9a. Confusion Matrices

In [97]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    # Confusion matrix
    confusion_matrices[model_name] = multilabel_confusion_matrix(ytrue, yhat)
    print(f"{model_name} confusion matrix: {os.linesep}{confusion_matrices[model_name]}")

# Collect results 
eval_results['confusion matrix'] = confusion_matrices

LSTM_new confusion matrix: 
[[[45  1]
  [ 1 12]]

 [[49  1]
  [ 2  7]]

 [[46  7]
  [ 1  5]]

 [[42  1]
  [ 3 13]]

 [[43  1]
  [ 4 11]]]
LSTM_Attention_new confusion matrix: 
[[[45  1]
  [ 1 12]]

 [[50  0]
  [ 0  9]]

 [[51  2]
  [ 2  4]]

 [[43  0]
  [ 0 16]]

 [[42  2]
  [ 2 13]]]


## 9b. Accuracy

In [29]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    # Model accuracy
    classification_accuracies[model_name] = accuracy_score(ytrue, yhat)    
    print(f"{model_name} classification accuracy = {round(classification_accuracies[model_name]*100,3)}%")

# Collect results 
eval_results['accuracy'] = classification_accuracies

LSTM_new classification accuracy = 20.69%
LSTM_Attention_new classification accuracy = 100.0%


## 9c. Precision, Recall, and F1 Score

In [98]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    print(y_test.shape)
    print(yhat)

    # Precision, recall, and f1 score
    report = classification_report(ytrue, yhat, target_names=actions, output_dict=True)
    
    precisions[model_name] = report['weighted avg']['precision']
    recalls[model_name] = report['weighted avg']['recall']
    f1_scores[model_name] = report['weighted avg']['f1-score'] 
   
    print(f"{model_name} weighted average precision = {round(precisions[model_name],3)}")
    print(f"{model_name} weighted average recall = {round(recalls[model_name],3)}")
    print(f"{model_name} weighted average f1-score = {round(f1_scores[model_name],3)}\n")

# Collect results 
eval_results['precision'] = precisions
eval_results['recall'] = recalls
eval_results['f1 score'] = f1_scores

(59, 5)
[0, 4, 3, 0, 3, 2, 0, 2, 0, 0, 3, 2, 3, 3, 4, 0, 2, 4, 4, 2, 4, 4, 2, 3, 0, 4, 3, 0, 3, 3, 0, 3, 1, 2, 3, 3, 2, 4, 1, 0, 1, 1, 0, 3, 1, 1, 0, 0, 4, 2, 4, 2, 4, 4, 2, 2, 1, 1, 3]
LSTM_new weighted average precision = 0.864
LSTM_new weighted average recall = 0.814
LSTM_new weighted average f1-score = 0.828

(59, 5)
[0, 4, 4, 0, 3, 2, 0, 1, 2, 0, 3, 1, 3, 3, 4, 0, 3, 4, 4, 4, 4, 4, 4, 3, 0, 4, 3, 0, 3, 3, 0, 3, 0, 2, 3, 3, 3, 4, 1, 0, 1, 1, 0, 3, 1, 1, 2, 0, 4, 4, 0, 2, 3, 4, 2, 4, 1, 1, 3]
LSTM_Attention_new weighted average precision = 0.915
LSTM_Attention_new weighted average recall = 0.915
LSTM_Attention_new weighted average f1-score = 0.915



# 10. Choose Model to Test in Real Time

In [99]:
model = AttnLSTM
model_name = 'AttnLSTM'

# 11. Calculate Joint Angles & Count Reps

In [100]:
def calculate_angle(a,b,c):
    """
    Computes 3D joint angle inferred by 3 keypoints and their relative positions to one another
    
    """
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [102]:
def get_coordinates(landmarks, mp_pose, side, joint):
    """
    Retrieves x and y coordinates of a particular keypoint from the pose estimation model
         
     Args:
         landmarks: processed keypoints from the pose estimation model
         mp_pose: Mediapipe pose estimation model
         side: 'left' or 'right'. Denotes the side of the body of the landmark of interest.
         joint: 'shoulder', 'elbow', 'wrist', 'hip', 'knee', or 'ankle'. Denotes which body joint is associated with the landmark of interest.
    
    """
    coord = getattr(mp_pose.PoseLandmark,side.upper()+"_"+joint.upper())
    x_coord_val = landmarks[coord.value].x
    y_coord_val = landmarks[coord.value].y
    return [x_coord_val, y_coord_val]            

In [101]:
def viz_joint_angle(image, angle, joint):
    """
    Displays the joint angle value near the joint within the image frame
    
    """
    cv2.putText(image, str(int(angle)), 
                   tuple(np.multiply(joint, [640, 480]).astype(int)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
    return

In [122]:
def count_reps(image, current_action, landmarks, mp_pose):
    """
    Counts repetitions of each exercise. Global count and stage (i.e., state) variables are updated within this function.
    
    """

    global curl_counter, press_counter, squat_counter,butbrdige_counter,leglift_counter, curl_stage, press_stage, squat_stage, butbrdige_stage, leglift_stage,r_leglift_stage,l_butbrdige_stage

    if current_action == 'butbrdige':
        shoulder_l = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        hip_l = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        knee_l = get_coordinates(landmarks, mp_pose, 'left', 'knee')

        shoulder_r = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        hip_r = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        knee_r = get_coordinates(landmarks, mp_pose, 'right', 'knee')

        angle_left = calculate_angle(shoulder_l,hip_l,knee_l)
        angle_right = calculate_angle(knee_r,hip_r,shoulder_r)
        
        # if (angle_right <180 and angle_right > 170):
        #     butbrdige_stage="up"
        #     print(butbrdige_stage,"right")
        # if (angle_right <160 and butbrdige_stage=="up"):
        #     print("ddddddd_right",butbrdige_stage)
        #     butbrdige_stage="down"
        #     butbrdige_counter+=1

        if(angle_left <180 and angle_left > 170) :
            l_butbrdige_stage="up"
            print(l_butbrdige_stage,"Left")
        if(angle_left <160 and l_butbrdige_stage=="up"):
            print("ddddddd_left",l_butbrdige_stage)
            l_butbrdige_stage="down"
            butbrdige_counter+=1

        press_stage = None
        squat_stage = None
        curl_stage=None
        leglift_stage=None
        r_leglift_stage=None

        
        viz_joint_angle(image, angle_right, hip_r)

    elif current_action == 'leglift':

        shoulder_l = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        hip_l = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        knee_l = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        ankle_l = get_coordinates(landmarks, mp_pose, 'left', 'ankle')

        shoulder_r = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        hip_r = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        knee_r = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        ankle_r = get_coordinates(landmarks, mp_pose, 'right', 'ankle')

        hip_angle_left = calculate_angle(shoulder_l,hip_l,knee_l)
        hip_angle_right = calculate_angle(shoulder_r,hip_r,knee_r)

        knee_angle_left = calculate_angle(hip_l,knee_l,ankle_l)
        knee_angle_right = calculate_angle(hip_r,knee_r,ankle_r)
        
        d_hip_angle_left = calculate_angle(shoulder_l,hip_l,ankle_l)
        d_hip_angle_right = calculate_angle(shoulder_r,hip_r,ankle_r)
        # print("hip_angle_left",hip_angle_left)
        # print("knee_angle_left",knee_angle_left)

        if (hip_angle_left<100 and hip_angle_left > 70) and (knee_angle_left> 40 and knee_angle_left < 90):
            leglift_stage = "up" 
            # print(leglift_stage)
            # print("hip_angle_left",hip_angle_left)
            # print("knee_angle_left",knee_angle_left)
        if d_hip_angle_left>170 and leglift_stage == "up" :
            leglift_stage="down"
            leglift_counter +=1

        if (hip_angle_right<100 and hip_angle_right > 70) and (knee_angle_right> 40 and knee_angle_right < 90):
            r_leglift_stage = "up" 
            viz_joint_angle(image, hip_angle_right, hip_r)
            viz_joint_angle(image, knee_angle_right, knee_r)
        if d_hip_angle_right>170 and r_leglift_stage == "up" :
            r_leglift_stage="down"
            leglift_counter +=1

        
        butbrdige_stage=None
        press_stage = None
        squat_stage = None
        curl_stage = None
        l_butbrdige_stage =None
    
    elif current_action == 'curl':
        # Get coords
        shoulder_l = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow_l = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist_l = get_coordinates(landmarks, mp_pose, 'left', 'wrist')

        shoulder_r = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        elbow_r = get_coordinates(landmarks, mp_pose, 'right', 'elbow')
        wrist_r = get_coordinates(landmarks, mp_pose, 'right', 'wrist')
        
        # calculate elbow angle
        elbow_angle_l = calculate_angle(shoulder_l, elbow_l, wrist_l)
        elbow_angle_r = calculate_angle(shoulder_r, elbow_r, wrist_r)
        
        # curl counter logic
        if elbow_angle_l < 30 and elbow_angle_r<30:
            curl_stage = "up" 
        if elbow_angle_l > 160 and elbow_angle_r>160 and curl_stage =='up':
            curl_stage="down"  
            curl_counter +=1

        butbrdige_stage=None
        press_stage = None
        squat_stage = None
        leglift_stage=None
        r_leglift_stage=None
        l_butbrdige_stage =None
            
        # Viz joint angle
        viz_joint_angle(image, elbow_angle_l, elbow_l)
        viz_joint_angle(image, elbow_angle_r, elbow_r)
        
    elif current_action == 'press':
        
        # Get coords
        shoulder_l = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow_l = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist_l = get_coordinates(landmarks, mp_pose, 'left', 'wrist')

        shoulder_r = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        elbow_r = get_coordinates(landmarks, mp_pose, 'right', 'elbow')
        wrist_r = get_coordinates(landmarks, mp_pose, 'right', 'wrist')

        # Calculate elbow angle
        elbow_angle_l = calculate_angle(shoulder_l, elbow_l, wrist_l)
        elbow_angle_r = calculate_angle(shoulder_r, elbow_r, wrist_r)
        
        # Compute distances between joints
       # shoulder2elbow_dist = abs(math.dist(shoulder,elbow))
        #shoulder2wrist_dist = abs(math.dist(shoulder,wrist))
        
        # Press counter logic
        #if (elbow_angle > 140) and (shoulder2elbow_dist < shoulder2wrist_dist):
        if (elbow_angle_l > 140) and (elbow_angle_l > 140) :
            press_stage = "up"
        #if (elbow_angle < 50) and (shoulder2elbow_dist > shoulder2wrist_dist) and (press_stage =='up')
        if (elbow_angle_l< 50 and press_stage =='up') and (elbow_angle_r< 50 and press_stage =='up'):
            press_stage='down'
            press_counter += 1
        butbrdige_stage=None
        curl_stage = None
        squat_stage = None
        leglift_stage=None
        r_leglift_stage=None
        l_butbrdige_stage =None
            
        viz_joint_angle(image, elbow_angle_l, elbow_l)
        viz_joint_angle(image, elbow_angle_r, elbow_r)
        
    elif current_action == 'squat':
        # Get coords
        # left side
        left_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        left_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        left_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        left_ankle = get_coordinates(landmarks, mp_pose, 'left', 'ankle')
        # right side
        right_shoulder = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        right_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        right_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        right_ankle = get_coordinates(landmarks, mp_pose, 'right', 'ankle')
        
        # Calculate knee angles
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        # Calculate hip angles
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        
        # Squat counter logic
        thr = 165
        if (left_knee_angle < thr) and (right_knee_angle < thr) and (left_hip_angle < thr) and (right_hip_angle < thr):
            squat_stage = "down"
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (squat_stage =='down'):
            squat_stage='up'
            squat_counter += 1
            
        butbrdige_stage=None
        curl_stage = None
        press_stage = None
        leglift_stage=None
        r_leglift_stage=None
        l_butbrdige_stage =None
            
        # Viz joint angles
        viz_joint_angle(image, left_knee_angle, left_knee)
        viz_joint_angle(image, left_hip_angle, left_hip)
        
    else:
        pass

# 12. Test in Real Time

In [123]:
def prob_viz(res, actions, input_frame, colors):
    """
    This function displays the model prediction probability distribution over the set of exercise classes
    as a horizontal bar graph
    
    """
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):        
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        
    return output_frame

In [124]:
# 1. New detection variables
sequence = []
predictions = []
res = []
threshold = 0.5 # minimum confidence to classify as an action/exerciseq
current_action = ''

# Rep counter logic variables
curl_counter = 0
press_counter = 0
squat_counter = 0
butbrdige_counter=0
leglift_counter=0

curl_stage = None
press_stage = None
squat_stage = None
butbrdige_stage= None
leglift_stage=None
l_butbrdige_stage=None

# Camera object
cap = cv2.VideoCapture(0)

# Video writer object that saves a video of the real time test
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') # video compression format
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 

video_name = os.path.join(os.getcwd(),f"{model_name}_real_time_test.avi")
out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), FPS, (WIDTH,HEIGHT))

# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detection
        image, results = mediapipe_detection(frame, pose)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)        
        sequence.append(keypoints)      
        sequence = sequence[-sequence_length:]
              
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]           
            predictions.append(np.argmax(res))
            current_action = actions[np.argmax(res)]
            confidence = np.max(res)
            
        #3. Viz logic
            # Erase current action variable if no probability is above threshold
            if confidence < threshold:
                current_action = ''

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
            # Count reps
            try:
                landmarks = results.pose_landmarks.landmark
                count_reps(
                    image, current_action, landmarks, mp_pose)
            except:
                pass

            # Display graphical information
            cv2.rectangle(image, (0,0), (1400, 40), colors[np.argmax(res)], -1)
            cv2.putText(image, 'curl ' + str(curl_counter), (5,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, 'press ' + str(press_counter), (225,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, 'squat ' + str(squat_counter), (450,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, 'butbrdige ' + str(butbrdige_counter), (625,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, 'leglift ' + str(leglift_counter), (825,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            
         
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Write to video file
        if ret == True:
            out.write(image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

up Left
up Left
up Left
up Left
up Left
up Left
up Left
ddddddd_left up
up Left
up Left
up Left
up Left
ddddddd_left up
up Left
up Left
up Left
up Left
ddddddd_left up
up Left
up Left
up Left
up Left
up Left
ddddddd_left up
up Left
up Left
up Left
up Left
up Left
up Left
up Left
ddddddd_left up


: 

In [62]:
cap.release()
out.release()
cv2.destroyAllWindows()

: 